In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Train.csv")

columns_to_keep = [
    'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 
    'DeviceId', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 
    'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 
    'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 
    'State', 'City'
]

df=df.loc[:, columns_to_keep]

df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})

# Save the DataFrame to a CSV file
df.to_csv('Processed_GUIDE_Train.csv', index=False)


In [29]:

import pandas as pd

df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Test.csv")

columns_to_keep = [
    'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 
    'DeviceId', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 
    'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 
    'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 
    'State', 'City'
]

df=df.loc[:, columns_to_keep]

df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})

# Save the DataFrame to a CSV file
df.to_csv('Processed_GUIDE_Test.csv', index=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_10472\1466830545.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Test.csv")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_10472\1466830545.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})


<!-- *Model* -->

In [20]:
import pandas as pd
df = pd.read_csv('Processed_GUIDE_Test.xls')

def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

def remove_outliers_iqr(df,column):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    IQR = q3 - q1
    lower_bound  = q1 - 1.5 * IQR
    upper_bound = q3 + 1.5 * IQR
    return df.loc[(df[column] > lower_bound) & (df[column] < upper_bound)]

df = remove_duplicates(df)

# columns_to_check = ['IncidentId', 'AlertId', 'DetectorId', 'DeviceId',
#                      'ApplicationName', 'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName']
# for column in columns_to_check:
#     df = remove_outliers_iqr(df, column)

# print("Data after removing duplicates and outliers:")
df.to_csv('Processed_GUIDE_Test.csv', index=False)
    

In [22]:
df = pd.read_csv('Processed_GUIDE_Train.xls')
df = remove_duplicates(df)
df.to_csv('Processed_GUIDE_Train.csv', index=False)

**RandomForestClassifier**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Load DataFrames
train_df = pd.read_csv('Processed_GUIDE_Train.csv')
test_df = pd.read_csv('Processed_GUIDE_Test.csv')

# Drop rows with NaNs in IncidentGrade
train_df = train_df.dropna(subset=['IncidentGrade'])

# Process categorical columns in training DataFrame
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols_train:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = le.fit_transform(train_df[col])

# Process categorical columns in test DataFrame
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    test_df[col] = test_df[col].astype(str)
    test_df[col] = le.fit_transform(test_df[col])

# Split train data into feature and target
X_train = train_df.drop('IncidentGrade', axis=1)
y_train = train_df['IncidentGrade']

X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Precision: 0.86
Recall: 0.84
F1 Score: 0.85


**Decision Tree Classifier**

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Load DataFrames
train_df = pd.read_csv('Processed_GUIDE_Train.csv')
test_df = pd.read_csv('Processed_GUIDE_Test.csv')

# Drop rows with NaNs in IncidentGrade
train_df = train_df.dropna(subset=['IncidentGrade'])

# Process categorical columns in training DataFrame
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols_train:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = le.fit_transform(train_df[col])

# Process categorical columns in test DataFrame
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    test_df[col] = test_df[col].astype(str)
    test_df[col] = le.fit_transform(test_df[col])

# Split train data into feature and target
X_train = train_df.drop('IncidentGrade', axis=1)
y_train = train_df['IncidentGrade']

X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Decision Tree Classifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Precision: 0.78
Recall: 0.78
F1 Score: 0.78


**Neural Network Model**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [4]:
# Load DataFrames
train_df = pd.read_csv('Processed_GUIDE_Train.csv')
test_df = pd.read_csv('Processed_GUIDE_Test.csv')

# Drop rows with NaNs in IncidentGrade
train_df = train_df.dropna(subset=['IncidentGrade'])

# Process categorical columns in training DataFrame
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols_train:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = le.fit_transform(train_df[col])

# Process categorical columns in test DataFrame
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    test_df[col] = test_df[col].astype(str)
    test_df[col] = le.fit_transform(test_df[col])

# Split train data into feature and target
X_train = train_df.drop('IncidentGrade', axis=1)
y_train = train_df['IncidentGrade']

X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert target to categorical
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)


# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
recall = recall_score(y_test_classes, y_pred_classes, average='macro')
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


C:\Users\ASUS\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 411s 2ms/step - accuracy: 0.7102 - loss: 0.6561 - val_accuracy: 0.7691 - val_loss: 0.5391
Epoch 2/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 457s 2ms/step - accuracy: 0.7704 - loss: 0.5356 - val_accuracy: 0.7793 - val_loss: 0.5175
Epoch 3/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 645s 3ms/step - accuracy: 0.7799 - loss: 0.5137 - val_accuracy: 0.7832 - val_loss: 0.5064
Epoch 4/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 478s 2ms/step - accuracy: 0.7844 - loss: 0.5037 - val_accuracy: 0.7874 - val_loss: 0.4953
Epoch 5/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 467s 2ms/step - accuracy: 0.7876 - loss: 0.4978 - val_accuracy: 0.7897 - val_loss: 0.4908
Epoch 6/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 585s 2ms/step - accuracy: 0.7900 - loss: 0.4925 - val_accuracy: 0.7939 - val_loss: 0.4870
Epoch 7/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 699s 3ms/step - accuracy: 0.7920 - loss: 0.4891 - val_accuracy: 0.7939 - val_loss: 0.4853
Epoch 8/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 68

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



236638/236638 ━━━━━━━━━━━━━━━━━━━━ 474s 2ms/step - accuracy: 0.7940 - loss: 0.4846 - val_accuracy: 0.7962 - val_loss: 0.4808
Epoch 10/10
236638/236638 ━━━━━━━━━━━━━━━━━━━━ 495s 2ms/step - accuracy: 0.7945 - loss: 0.4832 - val_accuracy: 0.7959 - val_loss: 0.4830
129625/129625 ━━━━━━━━━━━━━━━━━━━━ 183s 1ms/step
Precision: 0.74
Recall: 0.69
F1 Score: 0.71


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize model and scaler
model = RandomForestClassifier()
scaler = StandardScaler()
le = LabelEncoder()

# Process chunks
chunk_size = 10000
for chunk in pd.read_csv('Processed_GUIDE_Train.csv', chunksize=chunk_size):
    chunk = chunk.dropna(subset=['IncidentGrade'])
    for col in chunk.select_dtypes(include=['object']).columns:
        chunk[col] = le.fit_transform(chunk[col].astype(str))
    
    X_train = chunk.drop('IncidentGrade', axis=1)
    y_train = chunk['IncidentGrade']
    
    X_train = scaler.fit_transform(X_train)
    model.fit(X_train, y_train)

# Evaluate on test data
test_df = pd.read_csv('Processed_GUIDE_Test.csv')
test_df = test_df.dropna(subset=['IncidentGrade'])
for col in test_df.select_dtypes(include=['object']).columns:
    test_df[col] = le.fit_transform(test_df[col].astype(str))

X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']
X_test = scaler.transform(X_test)

y_pred = model.predict(X_test)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Load DataFrames
train_df = pd.read_csv('Processed_GUIDE_Train.csv')
test_df = pd.read_csv('Processed_GUIDE_Test.csv')

# Drop rows with NaNs in IncidentGrade
train_df = train_df.dropna(subset=['IncidentGrade'])

In [6]:
# Process categorical columns in training DataFrame
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols_train:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = le.fit_transform(train_df[col])

# Process categorical columns in test DataFrame
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    test_df[col] = test_df[col].astype(str)
    test_df[col] = le.fit_transform(test_df[col])

In [8]:
X_train = train_df.drop('IncidentGrade', axis=1)
y_train = train_df['IncidentGrade']

In [10]:
X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Train.csv")
columns_to_keep = [
    'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 
    'DeviceId', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 
    'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 
    'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 
    'State', 'City'
]

df=df.loc[:, columns_to_keep]

df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})

sample_df = df.sample(n=100000, random_state=42)
sample_df.to_csv('Sample_GUIDE_Train.csv', index=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14832\2651829891.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})


In [5]:
import pandas as pd

df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Test.csv")

columns_to_keep = [
    'IncidentId', 'AlertId', 'Timestamp', 'DetectorId', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType', 'EvidenceRole', 
    'DeviceId', 'IpAddress', 'Url', 'AccountSid', 'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName', 
    'NetworkMessageId', 'RegistryKey', 'RegistryValueName', 'RegistryValueData', 'ApplicationId', 'ApplicationName', 
    'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName', 'OSFamily', 'OSVersion', 'CountryCode', 
    'State', 'City'
]

df=df.loc[:, columns_to_keep]

df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})

# Save the DataFrame to a CSV file
sample_df = df.sample(n=100000, random_state=42)
sample_df.to_csv('Sample_GUIDE_Test.csv', index=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14832\248290320.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\MD112\Capstone\Microsoft\GUIDE_Test.csv")
C:\Users\ASUS\AppData\Local\Temp\ipykernel_14832\248290320.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['IncidentGrade'] = df['IncidentGrade'].replace({'BenignPositive': 1, 'FalsePositive': 2, 'TruePositive': 3})


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

# Load DataFrames
train_df = pd.read_csv('Sample_GUIDE_Train.csv')
test_df = pd.read_csv('Sample_GUIDE_Test.csv')

# Drop rows with NaNs in IncidentGrade
train_df = train_df.dropna(subset=['IncidentGrade'])

# Process categorical columns in training DataFrame
categorical_cols_train = train_df.select_dtypes(include=['object']).columns
le = LabelEncoder()
for col in categorical_cols_train:
    train_df[col] = train_df[col].astype(str)
    train_df[col] = le.fit_transform(train_df[col])

# Process categorical columns in test DataFrame
categorical_cols_test = test_df.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    test_df[col] = test_df[col].astype(str)
    test_df[col] = le.fit_transform(test_df[col])

# Split train data into feature and target
X_train = train_df.drop('IncidentGrade', axis=1)
y_train = train_df['IncidentGrade']

X_test = test_df.drop('IncidentGrade', axis=1)
y_test = test_df['IncidentGrade']

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train a Random Forest Classifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

Precision: 0.83
Recall: 0.80
F1 Score: 0.81


In [2]:
import pandas as pd

# Load DataFrames
train_df = pd.read_csv('Processed_GUIDE_Train.xls')
test_df = pd.read_csv('Processed_GUIDE_Test.xls')

# Sample a subset of the data to avoid MemoryError
train_df = train_df.sample(n=100000, random_state=42)
test_df = test_df.sample(n=100000, random_state=42)

In [ ]:
print(train_df)
print(test_df)



         IncidentId  AlertId            Timestamp  DetectorId  AlertTitle  \
8036545       49334   360163  2024-06-11 18:50:05          10           8   
3828916       52272    14178  2024-06-09 08:02:17           1           1   
4404679       21863    14266  2024-06-13 13:54:51          18          14   
8854100        3044     2411  2024-06-06 07:11:21          70        1871   
7977240         155   402605  2024-06-09 04:02:36           4           3   
...             ...      ...                  ...         ...         ...   
1259055      106728   444574  2024-06-03 12:33:55         867       53051   
8211233        1245   547267  2024-06-02 01:10:23          14          12   
6571643        1161   191438  2024-06-03 23:23:54           3           4   
1356383      539055   549552  2024-06-03 05:38:33           5          34   
3825628       72340    95117  2024-06-11 15:28:42          13          11   

                   Category  IncidentGrade   EntityType EvidenceRole  \
803

In [8]:
# For train_df
unique_train = train_df['IncidentGrade'].unique()
count_train = train_df['IncidentGrade'].value_counts()

print("Unique values in train_df['IncidentGrade']:")
print(unique_train)
print("\nCounts of unique values in train_df['IncidentGrade']:")
print(count_train)

# For test_df
unique_test = test_df['IncidentGrade'].unique()
count_test = test_df['IncidentGrade'].value_counts()

print("\nUnique values in test_df['IncidentGrade']:")
print(unique_test)
print("\nCounts of unique values in test_df['IncidentGrade']:")
print(count_test)


Unique values in train_df['IncidentGrade']:
[ 1.  2.  3. nan]

Counts of unique values in train_df['IncidentGrade']:
IncidentGrade
1.0    42822
3.0    35105
2.0    21505
Name: count, dtype: int64

Unique values in test_df['IncidentGrade']:
[1 3 2]

Counts of unique values in test_df['IncidentGrade']:
IncidentGrade
1    42297
3    36049
2    21654
Name: count, dtype: int64
